In [1]:
# Allow to import without installing
import sys
sys.path.insert(0, "..")

# Example files

These are downloaded from ???

In [4]:
import os, lzma
#basedir = os.path.join("/media/disk", "OSM_Data")
basedir = os.path.join("f:\\", "OSM_Data")
filename = "isle-of-wight-latest.osm.xz"
with lzma.open(os.path.join(basedir, filename), mode="rt", encoding="utf-8") as f:
    print(next(f), end="")
    print(next(f), end="")
    print(next(f), end="")
    print(next(f), end="")

<?xml version='1.0' encoding='UTF-8'?>
<osm version="0.6" generator="osmconvert 0.8.5" timestamp="2017-04-25T20:43:28Z">
	<bounds minlat="50.50555" minlon="-1.659074" maxlat="50.80102" maxlon="-1.0313699"/>
	<node id="195206" lat="50.6275781" lon="-1.1730057" version="10" timestamp="2016-03-29T12:53:40Z" changeset="38143882" uid="3099236" user="iwhs"/>


# Look at the generated data

In [3]:
import osmdigest.detail as detail

In [16]:
possible_node_tags = set()
possible_way_tags = set()
possible_relation_tags = set()

#with detail.Parser(os.path.join(basedir, filename)) as gen:
with detail.Parser("isle-of-wight-latest.osm") as gen:
    for x in gen:
        if isinstance(x, detail.OSM) or isinstance(x, detail.Bounds):
            print(x)
        elif isinstance(x, detail.Node):
            for y in x.subobjs:
                assert isinstance(y, detail.Tag)
                possible_node_tags.add(y.key)
        elif isinstance(x, detail.Way):
            for y in x.subobjs:
                if isinstance(y, detail.Tag):
                    possible_way_tags.add(y.key)
                else:
                    assert isinstance(y, detail.NodeRef)
        elif isinstance(x, detail.Relation):
            for y in x.subobjs:
                if isinstance(y, detail.Tag):
                    possible_relation_tags.add(y.key)
                else:
                    assert isinstance(y, detail.Member)
                    assert y.type in {"way", "node", "relation"}
        else:
            raise Exception("Should see this")

OSM(version=0.6, generator=osmconvert 0.8.5, timestamp=2017-04-25 20:43:28)
Bounds(latitude:[50.50555,50.80102], longitude:[-1.659074,-1.0313699]


In [17]:
len(possible_node_tags), list(possible_node_tags)[:5]

(335,
 ['facebook',
  'seamark:light:3:period',
  'aeroway',
  'contact:website',
  'fuel:biodiesel'])

In [18]:
len(possible_way_tags), list(possible_way_tags)[:5]

(484, ['source:bridge', 'surface', 'building:levels', 'upload_tag', 'taxi'])

In [19]:
len(possible_relation_tags), list(possible_relation_tags)[:5]

(151, ['aeroway', 'contact:website', 'lanes', 'residential', 'surface'])

# Parse the data in a reduced way

In [5]:
import osmdigest.digest as digest

In [14]:
possible_node_tags = set()
possible_way_tags = set()
possible_relation_tags = set()

for x in digest.parse("isle-of-wight-latest.osm"):
    if isinstance(x, digest.OSM) or isinstance(x, digest.Bounds):
        print(x)
    elif isinstance(x, digest.Node):
        for key in x.tags.keys():
            possible_node_tags.add(key)
    elif isinstance(x, digest.Way):
        for key in x.tags.keys():
            possible_way_tags.add(key)
    elif isinstance(x, digest.Relation):
        for key in x.tags.keys():
            possible_relation_tags.add(key)

OSM(version=0.6, generator=osmconvert 0.8.5, timestamp=2017-04-25 20:43:28)
Bounds(latitude:[50.50555,50.80102], longitude:[-1.659074,-1.0313699]


KeyboardInterrupt: 

In [10]:
len(possible_node_tags), list(possible_node_tags)[:5]

(335,
 ['seamark:light:2:character',
  'dispensing',
  'traffic_calming_1',
  'note_1',
  'url'])

In [11]:
len(possible_way_tags), list(possible_way_tags)[:5]

(484,
 ['seamark:light:2:character',
  'building:roof:material',
  'date_off',
  'url',
  'note_1'])

In [12]:
len(possible_relation_tags), list(possible_relation_tags)[:5]

(151, ['date_off', 'url', 'ref', 'from', 'site'])